In [2]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


### Load vectorizers

In [22]:
infiles = ['parted-cls/orders_vectorizer30.joblib.pkl.0']

import os
from sklearn.externals import joblib
from lib.helpers.fileSplitter import cat_files

cat_files(infiles, 'cls/orders_vectorizer30.joblib.pkl')

filename = 'cls/orders_vectorizer30.joblib.pkl'
orders_vectorizer = joblib.load(filename)

infiles = ['parted-cls/values_vectorizer30.joblib.pkl.0']

cat_files(infiles, 'cls/values_vectorizer30.joblib.pkl')

filename = 'cls/values_vectorizer30.joblib.pkl'
values_vectorizer = joblib.load(filename)

### Load classifyers 

In [4]:
infiles = ['parted-cls/device_class_code_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/device_class_code_logreg_cls.joblib.pkl')
clf_device_class_code = joblib.load('cls/device_class_code_logreg_cls.joblib.pkl')

infiles = ['parted-cls/os_family_code_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/os_family_code_logreg_cls.joblib.pkl')
clf_os_family_code = joblib.load('cls/os_family_code_logreg_cls.joblib.pkl')

infiles = ['parted-cls/os_code_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/os_code_logreg_cls.joblib.pkl')
clf_os_code = joblib.load('cls/os_code_logreg_cls.joblib.pkl')

infiles = ['parted-cls/ua_class_code_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/ua_class_code_logreg_cls.joblib.pkl')
clf_ua_class_code = joblib.load('cls/ua_class_code_logreg_cls.joblib.pkl')

infiles = ['parted-cls/ua_family_code_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/ua_family_code_logreg_cls.joblib.pkl')
clf_ua_family_code = joblib.load('cls/ua_family_code_logreg_cls.joblib.pkl')

infiles = ['parted-cls/ua_version_logreg_cls.joblib.pkl.0']

cat_files(infiles, 'cls/ua_version_logreg_cls.joblib.pkl')
clf_ua_version = joblib.load('cls/ua_version_logreg_cls.joblib.pkl')

### Load test data

In [18]:
main_data = np.load('df/main2_data.npy').tolist()[:20000]
values_data = np.load('df/values2_data.npy').tolist()[:20000]
order_data = np.load('df/order2_data.npy').tolist()[:20000]

main_df = pd.DataFrame(main_data)

main_df

,User_Agent,device_class_code,ip,os_code,os_family_code,timestamp,ua_class_code,ua_family_code,ua_version
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,desktop,62.89.6.193,windows_7,windows,1488578454,browser,chrome,chrome56.0.2924.87
1,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:34.0) G...,desktop,62.210.80.74,windows_8_1,windows,1488578468,browser,firefox,firefox34.0
2,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,desktop,77.87.101.132,windows_7,windows,1488578472,browser,yandex-browser,yandex-browser17.1.1.1003
3,Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...,desktop,216.239.90.19,windows_xp,windows,1488578508,browser,firefox,firefox7.0.1
4,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,smartphone,37.19.72.34,android_6,android,1488578544,mobile_browser,mobile_samsung_browser,mobile_samsung_browser4.0
...,...,...,...,...,...,...,...,...,...
19995,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,desktop,85.26.232.62,windows_10,windows,1488703450,browser,chrome,chrome56.0.2924.87
19996,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,desktop,46.118.53.27,windows_10,windows,1488703452,browser,yandex-browser,yandex-browser17.3.0.1785
19997,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,desktop,195.70.209.78,windows_7,windows,1488703453,browser,firefox,firefox39.0
19998,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,desktop,37.151.196.120,windows_7,windows,1488703458,browser,amigo,amigo54.0.2840.191


In [23]:
important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}
important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

from lib.parsers.logParser import LogParser

l_parser = LogParser(log_folder='Logs/')

l_parser.reassign_orders_values(order_data, values_data)
X_test = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=False)

100%|██████████| 20000/20000 [00:00<00:00, 230988.68it/s]

Sparse dummy orders shape: 
(20000, 50)
Sparse dummy values shape: 
(20000, 516)


### Calculate scores

Примечание: Для Decision Tree в cross_val_score по умолчанию берется показатель 'Accuracy'

Поскольку 'Accuracy' для линейной регрессии линейный мы не будем считать на 3-х или 5-ти фолдах(долго), а просто возьмем от тренировочной выборки 'Accuracy'

In [132]:
thres = 0.01

**Browser (clf_ua_family_code)**

In [133]:
from lib.thresholdPredictions import ThresholdPredictions

pred = ThresholdPredictions(user_agent_list=clf_ua_family_code.classes_.tolist(), clf=clf_ua_family_code)
y_test_names, y_predicted, compare_answers, answers_count = pred.predict(X_test, main_df.ua_family_code, thres, sparce_y=False, single_labels=True)

20000it [00:01, 16369.04it/s]


In [134]:
compare_frame = pd.concat(
    [
        pd.DataFrame(y_test_names),
        y_predicted, 
        pd.DataFrame(compare_answers),  
        pd.DataFrame(answers_count)
    ], keys=['browser_name', 'predicted_browser_name', 'browser_name_correctness', 'browser_name_count'], axis=1, join='inner')

compare_frame

,browser_name,predicted_browser_name,browser_name_correctness,browser_name_count
,ua_family_code,0,0,0
0,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6
1,firefox,"[chrome, chrome_mobile, chromodo, firefox, ie,...",True,11
2,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6
3,firefox,[firefox],True,1
4,mobile_samsung_browser,"[amigo, chrome, chrome_mobile, mobile_samsung_...",True,6
...,...,...,...,...
19995,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6
19996,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6
19997,firefox,"[chrome, firefox, firefox_mobile, safari, safa...",True,5


Accuracy: $ACC = \frac{TP + TN}{P + N},\ \ \mathrm{where}\ \ P + N = length,\ \ TP = sum(True), \ \ TN = 0$

In [135]:
print('Accuracy: {}'.format(sum(compare_frame.browser_name_correctness[0])/20000))

Accuracy: 0.96125


**Browser + Browser version (clf_ua_family_code + clf_ua_version)**

In [136]:
pred = ThresholdPredictions(user_agent_list=clf_ua_version.classes_.tolist(), clf=clf_ua_version)
y_test_names, y_predicted, compare_answers, answers_count = pred.predict(X_test, main_df.ua_version, thres, sparce_y=False, single_labels=True)

20000it [00:00, 35208.96it/s]


In [137]:
compare_frame['browser_version'] = pd.DataFrame(y_test_names)
compare_frame['predicted_browser_version'] = y_predicted
compare_frame['browser_version_correctness'] = pd.DataFrame(compare_answers)
compare_frame['browser_version_count'] = pd.DataFrame(answers_count)

In [138]:
compare_frame

,browser_name,predicted_browser_name,browser_name_correctness,browser_name_count,browser_version,predicted_browser_version,browser_version_correctness,browser_version_count
,ua_family_code,0,0,0,,,,
0,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12
1,firefox,"[chrome, chrome_mobile, chromodo, firefox, ie,...",True,11,firefox34.0,"[chrome36.0.1985.125, chrome39.0.2171.71, chro...",True,19
2,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.1.1.1003,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12
3,firefox,[firefox],True,1,firefox7.0.1,[firefox7.0.1],True,1
4,mobile_samsung_browser,"[amigo, chrome, chrome_mobile, mobile_samsung_...",True,6,mobile_samsung_browser4.0,"[amigo47.5.2526.115, chrome49.0.2623.112, chro...",True,11
...,...,...,...,...,...,...,...,...
19995,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12
19996,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.3.0.1785,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",False,12
19997,firefox,"[chrome, firefox, firefox_mobile, safari, safa...",True,5,firefox39.0,"[firefox12.0, firefox15.0.1, firefox38.0, fire...",False,13


In [139]:
print('Browser version accuracy: {}'.format(sum(compare_frame.browser_version_correctness)/20000))
print('Conditional accuracy: {}'.format(sum(compare_frame.browser_name_correctness[0] & compare_frame.browser_version_correctness)/20000))

Browser version accuracy: 0.8155
Conditional accuracy: 0.8141


**Browser + Browser version + Platform (clf_ua_family_code + clf_ua_version + clf_os_family_code)**

In [140]:
pred = ThresholdPredictions(user_agent_list=clf_os_family_code.classes_.tolist(), clf=clf_os_family_code)
y_test_names, y_predicted, compare_answers, answers_count = pred.predict(X_test, main_df.os_family_code, thres, sparce_y=False, single_labels=True)

20000it [00:00, 36212.81it/s]


In [141]:
compare_frame['platform'] = pd.DataFrame(y_test_names)
compare_frame['predicted_platform'] = y_predicted
compare_frame['platform_correctness'] = pd.DataFrame(compare_answers)
compare_frame['platform_count'] = pd.DataFrame(answers_count)
compare_frame

,browser_name,predicted_browser_name,browser_name_correctness,browser_name_count,browser_version,predicted_browser_version,browser_version_correctness,browser_version_count,platform,predicted_platform,platform_correctness,platform_count
,ua_family_code,0,0,0,,,,,,,,
0,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2
1,firefox,"[chrome, chrome_mobile, chromodo, firefox, ie,...",True,11,firefox34.0,"[chrome36.0.1985.125, chrome39.0.2171.71, chro...",True,19,windows,"[android, chrome_os, ios, linux, osx, windows]",True,6
2,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.1.1.1003,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2
3,firefox,[firefox],True,1,firefox7.0.1,[firefox7.0.1],True,1,windows,[windows],True,1
4,mobile_samsung_browser,"[amigo, chrome, chrome_mobile, mobile_samsung_...",True,6,mobile_samsung_browser4.0,"[amigo47.5.2526.115, chrome49.0.2623.112, chro...",True,11,android,"[android, windows]",True,2
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2
19996,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.3.0.1785,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",False,12,windows,"[android, windows]",True,2
19997,firefox,"[chrome, firefox, firefox_mobile, safari, safa...",True,5,firefox39.0,"[firefox12.0, firefox15.0.1, firefox38.0, fire...",False,13,windows,"[ios, linux, osx, windows]",True,4


In [142]:
print('Platform accuracy: {}'.format(sum(compare_frame.platform_correctness)/20000))
print('Conditional accuracy: {}'.format(sum(compare_frame.browser_name_correctness[0] & compare_frame.browser_version_correctness & compare_frame.platform_correctness)/20000))

Platform accuracy: 0.9648
Conditional accuracy: 0.80905


**Browser + Browser version + Platform + Platform version (clf_ua_family_code + clf_ua_version + clf_os_family_code + clf_os_code)**

In [143]:
pred = ThresholdPredictions(user_agent_list=clf_os_code.classes_.tolist(), clf=clf_os_code)
y_test_names, y_predicted, compare_answers, answers_count = pred.predict(X_test, main_df.os_code, thres, sparce_y=False, single_labels=True)

20000it [00:00, 34139.20it/s]


In [144]:
compare_frame['platform_version'] = pd.DataFrame(y_test_names)
compare_frame['predicted_platform_version'] = y_predicted
compare_frame['platform_version_correctness'] = pd.DataFrame(compare_answers)
compare_frame['platform_version_count'] = pd.DataFrame(answers_count)
compare_frame

,browser_name,predicted_browser_name,browser_name_correctness,browser_name_count,browser_version,predicted_browser_version,browser_version_correctness,browser_version_count,platform,predicted_platform,platform_correctness,platform_count,platform_version,predicted_platform_version,platform_version_correctness,platform_version_count
,ua_family_code,0,0,0,,,,,,,,,,,,
0,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2,windows_7,"[android_4_2, android_4_4, android_5_0, androi...",True,10
1,firefox,"[chrome, chrome_mobile, chromodo, firefox, ie,...",True,11,firefox34.0,"[chrome36.0.1985.125, chrome39.0.2171.71, chro...",True,19,windows,"[android, chrome_os, ios, linux, osx, windows]",True,6,windows_8_1,"[chrome_os, ios_7, ios_8, ios_9, linux, osx_10...",True,13
2,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.1.1.1003,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2,windows_7,"[android_4_2, android_4_4, android_5_0, androi...",True,10
3,firefox,[firefox],True,1,firefox7.0.1,[firefox7.0.1],True,1,windows,[windows],True,1,windows_xp,[windows_xp],True,1
4,mobile_samsung_browser,"[amigo, chrome, chrome_mobile, mobile_samsung_...",True,6,mobile_samsung_browser4.0,"[amigo47.5.2526.115, chrome49.0.2623.112, chro...",True,11,android,"[android, windows]",True,2,android_6,"[android_4_1, android_4_2, android_4_4, androi...",True,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,chrome,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,chrome56.0.2924.87,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",True,12,windows,"[android, windows]",True,2,windows_10,"[android_4_2, android_4_4, android_5_0, androi...",True,10
19996,yandex-browser,"[amigo, chrome, chrome_mobile, opera, yandex-b...",True,6,yandex-browser17.3.0.1785,"[chrome55.0.2883.87, chrome56.0.2924.87, chrom...",False,12,windows,"[android, windows]",True,2,windows_10,"[android_4_2, android_4_4, android_5_0, androi...",True,10
19997,firefox,"[chrome, firefox, firefox_mobile, safari, safa...",True,5,firefox39.0,"[firefox12.0, firefox15.0.1, firefox38.0, fire...",False,13,windows,"[ios, linux, osx, windows]",True,4,windows_7,"[ios_5, linux_ubuntu, osx_10_7, osx_10_8, wind...",True,9


In [145]:
print('Platform accuracy: {}'.format(sum(compare_frame.platform_version_correctness)/20000))
print('Conditional accuracy: {}'.format(sum(compare_frame.browser_name_correctness[0] & compare_frame.browser_version_correctness & compare_frame.platform_correctness & compare_frame.platform_version_correctness)/20000))

Platform accuracy: 0.9445
Conditional accuracy: 0.79785


### Save and load compare dataframe

In [101]:
np.save('df/compare_dummy_classification.npy', np.array(compare_frame))
print("Compare frame was saved with size(Bytes): {}".format(os.stat('df/compare_dummy_classification.npy').st_size))

Compare frame was saved with size(Bytes): 5099513


In [102]:
compare_frame2 = pd.DataFrame(np.load('df/compare_dummy_classification.npy').tolist())

In [103]:
compare_frame2.columns = [
    'browser_name', 
    'predicted_browser_name', 
    'browser_name_correctness', 
    'browser_name_count', 
    'browser_version', 
    'predicted_browser_version', 
    'browser_version_correctness', 
    'browser_version_count', 
    'platform', 
    'predicted_platform', 
    'platform_correctness', 
    'platform_count',
    'platform_version', 
    'predicted_platform_version', 
    'platform_version_correctness', 
    'platform_version_count'
]

# Conclusion (Light Threshold)

|             Layer            | DecisionTree | Thresolded Logit(0.024072) | Thresolded Logit(0.00075) | Thresolded Logit(0.01) |
|---------------------------------------------------------------------|--------------|---------|---------|---------|
| Layer 0 (browser_name)                                              |      0.79    | 0.9349  | 0.97175 | 0.96125 |
| Layer 1 (browser_name, browser_version)                             |      0.69    | 0.76575 | 0.94205 | 0.8141  |
| Layer 2 (browser_name, browser_version, platform)                   |      0.64    | 0.75885 | 0.94055 | 0.80905 |
| Layer 3 (browser_name, browser_version, platform, platform_version) |      0.65    |  0.735  |  0.9391 | 0.79785 |